In [7]:

import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

print('done')

In [11]:
data_dir = '../input/2019-2nd-ml-month-with-kakr'

train_data_path = join(data_dir, 'train.csv')   # 훈련 데이터 경로
test_data_path = join(data_dir, 'test.csv')      # 테스트, 즉 submission 시 사용할 데이터 경로

print(train_data_path)
print(test_data_path)

In [12]:

# 데이터 불러오기

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)
print('train data dim : {}'.format(train.shape))
print('test data dim : {}'.format(test.shape))

In [13]:
# 전체 데이터의 자료형 확인

train.info()

In [14]:
# 문자열인 트레인과 테스트의 date 컬럼을 정수형으로 정리

train['date'] = train['date'].apply(lambda i: i[:6]).astype(int)
test['date'] = test['date'].apply(lambda i: i[:6]).astype(int)

train.head()

In [15]:
# 트레인 데이터를 한번 카피해두고 가격과 다른 피처들의 상관계수 확인 
train_copy = train.copy()
total_corr = train_copy.corr()
total_corr['price'].sort_values()

In [16]:
fig, ax = plt.subplots()
fig.set_size_inches(12,8)

df_heatmap = sns.heatmap(total_corr, vmin = -1, vmax = 1, cbar = True, fmt = '.2f', square = False, cmap = 'PiYG')

## Data preprocessing

In [17]:
# living과 living15가 다른 집의 재건축여부 확인
train_copy.loc[train_copy['sqft_living']!=train_copy['sqft_living15'],
               ['sqft_living', 'sqft_living15', 'yr_renovated']]

In [18]:
# lot과 lot15가 다른 집의 재건축여부 확인
train_copy.loc[train_copy['sqft_lot']!=train_copy['sqft_lot15'],
               ['sqft_lot', 'sqft_lot15', 'yr_renovated']]

In [19]:
# 침실, 화장실 수를 합한 공간 컬럼 생성

train_copy['rooms'] = train_copy.apply(
        lambda d: (d['bedrooms'] + d['bathrooms']), axis=1)
train_copy

#### Number of rooms

In [20]:
train_copy['rooms_ratio'] = train_copy.apply(
        lambda d: (d['rooms']) / d['sqft_living'], axis=1)
train_copy

In [21]:
# 쓸만한 상태가 되었는지 상관계수 확인

total_corr = train_copy.corr()
total_corr['price'].sort_values(ascending=False)

In [24]:
# 쓸만한 상태가 되었는지 상관계수 확인
total_corr = train_copy.corr()
total_corr['price'].sort_values(ascending=False)

In [25]:
# yr_renovated의 값이 존재하는 경우 yr_renovated를 yr_built로 대체

train_copy['yr_since'] = train_copy['yr_built'] # 새로운 컬럼 생성해서 built 값으로 채움
train_copy.loc[train_copy['yr_renovated'] != 0, 'yr_since'] = train_copy['yr_renovated']
# renovated 값이 존재하는 경우 대체
train_copy

In [26]:
# 쓸만한 상태가 되었는지 상관계수 확인 

total_corr = train_copy.corr()
total_corr['price'].sort_values(ascending=False)

In [27]:
train_copy['price_per_area'] = train_copy.apply(
        lambda d: (d['price'] / d['sqft_living']), axis=1)
train_copy

#### 위도 & 경도확인

In [28]:
sns.relplot(x = 'long', y = 'lat', hue = 'price_per_area', alpha = 0.7, height = 6, aspect = 1.8, data = train_copy)

# 지역에 따라 집값에 유의미한 차이가 있기는 한 것 같은데 
# 위경도의 특징을 한번에 나타내는 피처를 어떻게 뽑아내야 할지 모르겠다

In [50]:
# test의 id 컬럼은 따로 저장해두기
# train에만 있는 타겟인 price 컬럼은 따로 저장 후 삭제

sub_id = test['id']

y = train['price']
del train['price']

print(train.columns)

In [51]:
# 트레인 데이터 개수 확인 후 데이터 병합

#train_len = len(train_copy)
#union = pd.concat((train_copy, test), axis=0)

train_len = len(train)
union = pd.concat((train, test), axis=0)

print(len(train))
print(len(union))

union

In [52]:
# id를 index로 설정

union.set_index('id', inplace = True)
print(union.columns)
union

In [53]:
train['price'] = np.log1p(train['price'])
print(train['price'])

sns.kdeplot(train['price'])
plt.show()

In [54]:

fig, ax = plt.subplots(7, 3, figsize=(12, 20), constrained_layout=True) 

count = 0
columns = union.columns
for row in range(7):
    for col in range(3):
        sns.kdeplot(union[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == len(union.columns) :
            break

In [55]:
for col in union.columns:
    if union[col].skew() > 2:
        print('{:15}'.format(col), 'Skewness: {:05.2f}'.format(union[col].skew()))

# waterfront와 yr_renovated는 0이 많은 데이터이므로 로그변환x
# veiw도 범주형 데이터이므로 로그변환 x
# 그림으로는 sqft 가 모두 치우쳐보이는데 실제로 왜도가 크지는 않다

In [56]:
# 치우친 컬럼들을 skew_columns 리스트 안에 담고
# np.log1p()를 활용해서 로그 변환
# numpy.log1p() 함수는 각 요소에 대해 자연로그 log(1 + x)을 반환해 주는 함수

skew_columns = ['sqft_lot', 'sqft_lot15']

for c in skew_columns:
    union[c] = np.log1p(union[c].values)

print('done')

In [57]:

plt.figure(figsize=(8, 4))
plt.subplot(121),sns.kdeplot(union['sqft_lot'])
plt.subplot(122),sns.kdeplot(union['sqft_lot15'])

#### Bedrooms 이상치 확인

In [58]:
# 침실이 30개 초과인 집 확인

union.loc[union['bedrooms'] > 30, ['bedrooms']]

In [59]:
# bedrooms의 사분위수와, 주거면적당 침실의 개수 산점도 확인

plt.figure(figsize=(10, 5))
plt.subplot(121),sns.boxplot(y='bedrooms', data=union)
plt.subplot(122),sns.scatterplot(x = 'sqft_living', y = 'bedrooms', data=union)

In [60]:
# 전체 데이터 다시 분할

sub = union.iloc[train_len:, :] # 트레인 데이터 개수 이후부터는 타겟
x = union.iloc[:train_len, :] # 트레인 데이터 이전은 학습용

print(x.shape)
print(sub.shape)

#### check out target data

In [61]:

# y의 분포 확인

sns.kdeplot(y)
plt.show()

In [62]:
print('price Skewness: {:05.2f}'.format(y.skew()))

## 모델 제작

### LGBM

In [63]:
# 데이터셋 분할과 RMSE 점수를 계산하기 위한 모듈을 임포트

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

#RMSE 계산 함수
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

# RMSE 계산을 위한 함수 작성
# y_test나 y_pred는 로그변환된 값이므로 환산해줘야함

def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

print('done')

In [64]:
random_state=2020

gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = XGBRegressor(random_state=random_state)
lightgbm = LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

In [65]:
#각 모델들의 RMSE 확인
def get_scores(models, train, y):
    df = {}
    
    for model in models:
        model_name = model.__class__.__name__
        
        X_train, X_test, y_train, y_test = train_test_split(train, y, random_state=random_state, test_size=0.2)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        df[model_name] = rmse(y_test, y_pred)
        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
            
    return score_df

get_scores(models, train, y)

#### GridSearch

In [66]:
#grid search 함수
from sklearn.model_selection import GridSearchCV

print('done')

def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')

    return results

In [67]:

"""
다음과 같은 과정을 진행할 수 있는 `my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5)` 함수를 구현해 보세요.

1. GridSearchCV 모델로 `model`을 초기화합니다.
2. 모델을 fitting 합니다.
3. params, score에 각 조합에 대한 결과를 저장합니다. 
4. 데이터 프레임을 생성하고, RMSLE 값을 추가한 후 점수가 높은 순서로 정렬한 `results`를 반환합니다.
"""

def my_GridSearch(model, x, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(x, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')

    return results

print('done')

In [68]:
# 생성한 함수로 그리드 탐색

param_grid = {
    'learning_rate' : [0.034, 0.035, 0.036],
    'n_estimators': [930, 940, 950],
    'max_depth': [15, 16,17],
}

model = LGBMRegressor(random_state=random_state)
my_GridSearch(model, x, y, param_grid, verbose=2, n_jobs=5)

In [82]:
"""
아래의 과정을 수행하는 `save_submission(model, x, y, test, model_name, rmsle)` 함수를 구현해 주세요.
1. 모델을 `x`, `y`로 학습시킵니다.
2. `test`에 대해 예측합니다.
3. 예측값을 `np.expm1`으로 변환하고, `submission_model_name_RMSLE_100000.csv` 형태의 `csv` 파일을 저장합니다.
"""

def save_submission(model, x, y, test, model_name, rmsle=None):
    model.fit(x, y)
    prediction = model.predict(test)
    prediction = np.expm1(prediction)
    data_dir = os.getenv('HOME')+'/aiffel/EXP_06_kaggle_kakr_housing/data/'
    submission_path = join(data_dir, 'sample_submission.csv')
    submission = pd.read_csv(submission_path)
    submission['price'] = prediction
    submission_csv_path = 'sample_submission'
    submission.to_csv('sample_submission.csv')
    print('{} saved!'.format(submission_csv_path))

print('done')

In [84]:
save_submission(model, train, y, test, 'lgbm', rmsle='0.159795')